In [ ]:
# default_exp __init__

In [ ]:
# hide
import os
notebooks_dir = os.getcwd()
project_dir = os.path.dirname(notebooks_dir)

import sys
sys.path.append(project_dir)

# BookKeeper

In [ ]:
# export
class Portfolio(object):
    def __init__(self, portfolio):
        if isinstance(portfolio, type(self)):
            self.__dict__.update(portfolio.__dict__)
        if isinstance(portfolio, dict):
            self.__dict__.update(portfolio)
        if isinstance(portfolio, list):
            self.__dict__.update(dict(portfolio))

In [ ]:
from ccstabilizer import Binance
from ccstabilizer import MXC
from ccstabilizer import secrets

In [ ]:
# export
import copy
from datetime import datetime
from decimal import Decimal
import os


class BookKeeper(object):

    def __init__(self, status_list):
        self.binance = Binance(os.environ['BINANCE_API_KEY'], os.environ['BINANCE_API_SECRET'])
        self.mxc = MXC(os.environ['MXC_ACCESS_KEY'], os.environ['MXC_SECRET_KEY'])

        self.status_list = status_list
        self.portfolio = self.get_portfolio()
        for status in self.status_list:
            status.unused_fiat_money = getattr(self.portfolio, status.fiat_symbol)

        self.estimate_functions = {
            'buy': self.estimate_status_by_buying,
            'sell': self.estimate_status_by_selling
        }

    def __enter__(self):
        self.fsh = open('sample-history.txt', 'a', 1)
        self.fth = open('transaction-history.txt', 'a', 1)
#         self.fth_origin = open('transaction-history-origin.txt', 'a', 1)
        type(self).fth = open('transaction-exception.txt', 'a', 1)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.fsh.close()
        self.fth.close()
#         self.fth_origin.close()
        type(self).fth.close()
        if exc_type == Exception and str(exc_value) == MXC.ERROR_MESSAGES[30020]:
            return True
        return False

    def get_portfolio(self):
        return Portfolio([
            (symbol, Decimal(asset.get('available', '0'))) for symbol, asset in self.mxc.get_account_info().items()
        ])

    def estimate_status_list(self):
        old_portfolio = self.portfolio
        new_portfolio = self.get_portfolio()
        self.portfolio = new_portfolio

        new_status_list = []
        for status in self.status_list:
            new_status = type(self).estimate_status(status, old_portfolio, new_portfolio)
            new_status_list.append(new_status)
            if new_status is not status:
                self.fth.write(new_status.last_transaction)

        return new_status_list

    @classmethod
    def estimate_status(cls, status, old_portfolio, new_portfolio):
        old_fiat_money = getattr(old_portfolio, status.fiat_symbol)
        old_amount = getattr(old_portfolio, status.crypto_symbol, 0)
        new_fiat_money = getattr(new_portfolio, status.fiat_symbol)
        new_amount = getattr(new_portfolio, status.crypto_symbol, 0)

        if old_amount > new_amount and old_fiat_money < new_fiat_money:
            amount = old_amount - new_amount
            fiat_price = (new_fiat_money - old_fiat_money) / amount

            old_status_amount = status.bought_amount
            new_status = cls.estimate_status_by_selling(
                status = status,
                unit_amount = amount / status.trade_unit,
                fiat_price = fiat_price,
                buy_unit_amount = min(0, new_amount - old_status_amount) / status.trade_unit,
                new_unused_fiat_money = new_fiat_money
            )

            now_sell_fiat_price = status.now_sell_fiat_price
            if now_sell_fiat_price is None:
                type(self).fth.write(new_status.last_transaction)
                status.unused_fiat_money = new_fiat_money
                return status
            if abs(fiat_price - now_sell_fiat_price) / now_sell_fiat_price > Decimal(50/100):
                type(self).fth.write(new_status.last_transaction)
                status.unused_fiat_money = new_fiat_money
                return status

            return new_status

        if old_amount < new_amount and old_fiat_money > new_fiat_money:
            amount = new_amount - old_amount
            fiat_price = (old_fiat_money - new_fiat_money) / amount

            new_status = cls.estimate_status_by_buying(
                status = status,
                unit_amount = amount / status.trade_unit,
                fiat_price = fiat_price,
                new_unused_fiat_money = new_fiat_money
            )

            now_buy_fiat_price = status.now_buy_fiat_price
            if now_buy_fiat_price is None:
                type(self).fth.write(new_status.last_transaction)
                status.unused_fiat_money = new_fiat_money
                return status
            if abs(fiat_price - now_buy_fiat_price) / now_buy_fiat_price > Decimal(50) / 100:
                type(self).fth.write(new_status.last_transaction)
                status.unused_fiat_money = new_fiat_money
                return status

            return new_status

        if old_amount > new_amount and old_fiat_money >= new_fiat_money:
            amount = old_amount - new_amount
            old_status_amount = status.bought_amount
            return cls.estimate_status_by_utilizing(
                status = status,
                unit_amount = amount / status.trade_unit,
                buy_unit_amount = min(0, new_amount - old_status_amount) / status.trade_unit,
                new_unused_fiat_money = new_fiat_money
            )

        if old_amount < new_amount and old_fiat_money <= new_fiat_money:
            amount = new_amount - old_amount
            return cls.estimate_status_by_receiving(
                status = status,
                unit_amount = amount / status.trade_unit,
                new_unused_fiat_money = new_fiat_money
            )

        status.unused_fiat_money = new_fiat_money
        return status

    @staticmethod
    def estimate_status_by_receiving(status, unit_amount, new_unused_fiat_money=None):
        if unit_amount <= 0:
            raise Exception('estimate_status_by_receiving: unit_amount <= 0')

        old_status = status
        new_status = copy.deepcopy(old_status)

        amount = status.trade_unit * unit_amount

        old_amount = old_status.bought_amount
        old_percentage = old_status.get_usage() * 100
        new_amount = new_status.bought_amount
        new_percentage = new_status.get_usage(new_unused_fiat_money) * 100

        new_status.last_transaction = (
            f'RECEIVED {amount:.4f} = -{old_amount:.4f} ({old_percentage:.2f}%) +{new_amount:.4f} ({new_percentage:.2f}%) {amount:+.4f} {new_status.crypto_symbol}\n'
        )

        return new_status

    @staticmethod
    def estimate_status_by_utilizing(status, unit_amount, buy_unit_amount=None, new_unused_fiat_money=None):
        if unit_amount <= 0:
            raise Exception('estimate_status_by_utilizing: unit_amount <= 0')
        if buy_unit_amount is None:
            buy_unit_amount = -unit_amount

        old_status = status
        new_status = copy.deepcopy(old_status)

        new_status.update_bought_status(fiat_price=new_status.bought_average_fiat_price, unit_amount=buy_unit_amount)

        amount = status.trade_unit * unit_amount

        old_amount = old_status.bought_amount
        old_percentage = old_status.get_usage() * 100
        new_amount = new_status.bought_amount
        new_percentage = new_status.get_usage(new_unused_fiat_money) * 100

        new_status.last_transaction = (
            f'UTILIZED {amount:.4f} = +{old_amount:.4f} ({old_percentage:.2f}%) -{new_amount:.4f} ({new_percentage:.2f}%) {amount-(old_amount-new_amount):+.4f} {new_status.crypto_symbol}\n'
        )

        return new_status

    @staticmethod
    def estimate_status_by_buying(status, unit_amount, fiat_price=None, new_unused_fiat_money=None):
        if unit_amount <= 0:
            raise Exception('estimate_status_by_buying: unit_amount <= 0')
        if fiat_price is None:
            fiat_price = status.now_buy_fiat_price

        old_status = status
        new_status = copy.deepcopy(old_status)

        new_status.buy_count += 1
        new_status.update_bought_status(fiat_price=fiat_price, unit_amount=unit_amount)

        new_status.update(
            status = {
                'sold_unit_amount': 0
            }
        )

        amount = status.trade_unit * unit_amount

        old_amount = old_status.bought_amount
        old_percentage = old_status.get_usage() * 100
        new_amount = new_status.bought_amount
        new_percentage = new_status.get_usage(new_unused_fiat_money) * 100

        new_status.last_transaction = (
            f'BOUGHT {amount:.4f} = -{old_amount:.4f} ({old_percentage:.2f}%) +{new_amount:.4f} ({new_percentage:.2f}%) {new_status.crypto_symbol} for {fiat_price:.4f} {new_status.fiat_symbol}/{new_status.crypto_symbol}:'
            f' NO.{new_status.trade_count + 1}-{new_status.buy_count} at {datetime.now()}\n'
        )

        return new_status

    @staticmethod
    def estimate_status_by_selling(status, unit_amount, fiat_price=None, buy_unit_amount=None, new_unused_fiat_money=None):
        if unit_amount <= 0:
            raise Exception('estimate_status_by_selling: unit_amount <= 0')
        if fiat_price is None:
            fiat_price = status.now_sell_fiat_price
        if buy_unit_amount is None:
            buy_unit_amount = -unit_amount

        old_status = status
        new_status = copy.deepcopy(old_status)

        new_status.sell_count += 1
        new_status.update_bought_status(fiat_price=new_status.bought_average_fiat_price, unit_amount=buy_unit_amount)
        new_status.update_sold_status(fiat_price=fiat_price, unit_amount=unit_amount)

        if new_status.bought_unit_amount == 0:
            new_status.update(
                status = {
                    'trade_count': new_status.trade_count + 1,
                    'buy_count': 0,
                    'sell_count': 0
                }
            )

        amount = status.trade_unit * unit_amount

        old_avg_price = old_status.bought_average_fiat_price
        diff_price = fiat_price - old_avg_price
        gained_fiat_money = old_status.estimate_gained_fiat_money(unit_amount, fiat_price)
        new_status.total_gained_fiat_money += gained_fiat_money

        old_amount = old_status.bought_amount
        old_percentage = old_status.get_usage() * 100
        new_amount = new_status.bought_amount
        new_percentage = new_status.get_usage(new_unused_fiat_money) * 100

        new_status.last_transaction = (
            f'GAINED {gained_fiat_money:+.4f} {new_status.fiat_symbol}: GAINED {new_status.total_gained_fiat_money:+.4f} {new_status.fiat_symbol} in total at {datetime.now()} after\n'
            f'SOLD {amount:.4f} = +{old_amount:.4f} ({old_percentage:.2f}%) -{new_amount:.4f} ({new_percentage:.2f}%) {amount-(old_amount-new_amount):+.4f} {new_status.crypto_symbol}'
            f' for {diff_price:.4f} = {fiat_price:.4f} - {old_avg_price:.4f} {new_status.fiat_symbol}/{new_status.crypto_symbol}: NO.{new_status.trade_count + 1}-{new_status.sell_count}\n'
        )

        return new_status

In [ ]:
from ccstabilizer import Status
from decimal import Decimal

crypto_symbol = 'AR'
fiat_symbol = 'USDT'
crypto_info = {}

status = Status(
    robot_name=f'{crypto_symbol} Robot',
    crypto_symbol=crypto_symbol,
    fiat_symbol=fiat_symbol,
    max_used_fiat_money_limit=Decimal('1000'),
    **crypto_info
)

In [ ]:
BookKeeper(status_list=[status]).portfolio.__dict__

{'B20': Decimal('2.43'),
 'AR': Decimal('9.01'),
 'LOON': Decimal('1.96800109'),
 'OCEAN': Decimal('85.3'),
 'HOPR': Decimal('213.02'),
 'ETH': Decimal('3.322E-9'),
 'XLM': Decimal('0.0058'),
 'USDT': Decimal('934.58012431152'),
 'VELO': Decimal('273.64'),
 'SRK': Decimal('0'),
 'AUR': Decimal('0'),
 'BDP': Decimal('50.92')}